# AdventureWorks - Easy

![rel](https://sqlzoo.net/w/images/2/28/AdventureWorks.png)

In [1]:
import getpass
import psycopg2
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
pwd = getpass.getpass()
engine = create_engine(
    'postgresql+psycopg2://postgres:%s@192.168.31.31:15432/sqlzoo' % (pwd))
pd.set_option('display.max_rows', 100)

········


In [2]:
cust_aw = pd.read_sql_table('CustomerAW', engine)
cust_addr = pd.read_sql_table('CustomerAddress', engine)
addr = pd.read_sql_table('Address', engine)
product = pd.read_sql_table('Product', engine)
order_det = pd.read_sql_table('SalesOrderDetail', engine)
order_head = pd.read_sql_table('SalesOrderHeader', engine)

## 1.
**Show the first name and the email address of customer with CompanyName 'Bike World'**

In [3]:
cust_aw.loc[cust_aw['CompanyName']=='Bike World', 
            ['FirstName', 'EmailAddress']]

,FirstName,EmailAddress
16,Kerim,kerim0@adventure-works.com


## 2.
**Show the CompanyName for all customers with an address in City 'Dallas'.**

In [4]:
a = (cust_aw.merge(cust_addr, on='CustomerID')
     .merge(addr, on='AddressID'))
(a.loc[a['City']=='Dallas', ['CompanyName']].sort_values('CompanyName')
 .drop_duplicates())

,CompanyName
84,Elite Bikes
167,Rental Bikes
106,Third Bike Store
73,Town Industries
131,Unsurpassed Bikes


## 3.
**How many items with ListPrice more than $1000 have been sold?**

In [5]:
a = order_det.merge(product, on='ProductID').query('ListPrice>1000')
pd.DataFrame({'qty': [sum(a['OrderQty'])]})

,qty
0,451


## 4.
**Give the CompanyName of those customers with orders over $100000. Include the subtotal plus tax plus freight.**

In [6]:
a = (order_head.merge(cust_aw, on='CustomerID'))
a['n_orders'] = a['SubTotal'].fillna(0) + a['TaxAmt'].fillna(0) + \
    a['Freight'].fillna(0)

a.loc[a['n_orders']>100000, ['CompanyName']].drop_duplicates()

,CompanyName
5,Action Bicycle Specialists
29,Metropolitan Bicycle Supply


## 5.
**Find the number of left racing socks ('Racing Socks, L') ordered by CompanyName 'Riding Cycles'**

In [7]:
a = (order_det.merge(product, on='ProductID')
     .merge(order_head, on='SalesOrderID')
     .merge(cust_aw, on='CustomerID'))
pd.DataFrame({'total': [a.loc[(a['Name']=='Racing Socks, L') & 
                              (a['CompanyName']=='Riding Cycles'),
                              'OrderQty'].sum()]})

,total
0,3
